In [4]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker
import matplotlib.colors
import scipy.stats
import pandas
import itertools
from mpl_toolkits import basemap
import mpl_toolkits.axes_grid1
import random
import datetime
import time

degree_sign = u'\u00B0'
mp.rcParams.update({'mathtext.default': 'regular'})
mp.rcParams['hatch.color'] = '0.5'

% matplotlib inline

# Set up lat/lon of locations of interest

In [5]:
LA_lat = 34.0522
LA_lon = 118.2437 # deg west
LA_lon = 180. + (180-LA_lon)

Oroville_dam_lat = 39.5380
Oroville_dam_lon = 121.4831 # deg west
Oroville_dam_lon = 360 - Oroville_dam_lon

SF_lat = 37.7749
SF_lon = 122.4194
SF_lon = 360 - SF_lon

# Import coastal grid point indices

In [6]:
cali_coast_latlon_indices = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/coastal_latlon_array_indices_3x.npy')
cali_coast_latlon_indices_zip = [i for i in zip(cali_coast_latlon_indices[:,0], cali_coast_latlon_indices[:,1])]

cali_coast_latlon_indices_3d = cali_coast_latlon_indices.reshape((11,3,2))
#print(cali_coast_latlon_indices_3d[0,:,:])

# Specify return period information and percentiles

In [7]:
#return_period = 200 # in years
#events_per_year = 151-40

return_period = 25
events_per_year = 1

return_val_perc = 100*(1-1/(return_period*events_per_year))
print(return_val_perc)

96.0


In [8]:
window=30
half_window=int(window/2)

# Set up directories where data are stored

In [9]:
working_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/'
#save_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calculations/npy_files/'
#latlon_indices = numpy.load(working_dir + 'ccal_latlon_indices_array.npy'); region='ccal'
#latlon_indices = numpy.load(working_dir + 'ncal_latlon_indices_array.npy'); region='ncal'
#latlon_indices = numpy.load(working_dir + 'scal_latlon_indices_array.npy'); region='scal'

PRECT_lat = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lat.npy')
PRECT_lon = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lon.npy')

In [10]:
working_dir = '/Users/baird/google_drive/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/'
#threshold=0.0
threshold=0.1
#threshold=0.5
#threshold=1.0
#threshold=5.0
#threshold=10.0

# Import lat/lon information

In [11]:
PRECT_nlat = 26
PRECT_nlon = 25

latlon_indices = list(itertools.product(range(PRECT_nlat), range(PRECT_nlon)))
region = 'whole_domain'
window=1

# Open PIC data

In [12]:
year_start_pic = 402 #time_subsets[chunk,0]
year_end_pic = 2200 #time_subsets[chunk,1]

# create season strings
years_pic = numpy.arange(year_start_pic, year_end_pic+1, 1).astype(numpy.int)
#half_years_pic = numpy.arange(year_start_pic+0.75, year_end_pic, 1)
#season_strings_pic = numpy.empty(years.size-1, dtype=numpy.str)

season_strings_pic = [str(years_pic[i])+'-'+str(years_pic[i+1]) for i in range(years_pic.size-1)]
member_strings_pic = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_pic=year_end_pic-year_start_pic

# Save all events as a numpy array, and save return period values

# Open all PIC events (which have already been saved as a npy array)

In [13]:
all_events_pic = numpy.load('csv_files/all_events_pic.npy')

# Open return period calculations

In [14]:
return_period_values_pic_df = pandas.read_csv('csv_files/return_period_values_pic_df.csv')
return_period_values_pic = return_period_values_pic_df.values[:,1:]

# Open historical, RCP and place all data into a 3d array

In [15]:
ensemble_members = numpy.hstack((numpy.arange(1,36), numpy.arange(101,106)))
ensemble_names = ['{:03d}'.format(i) for i in ensemble_members]

In [16]:
year_start = 1920
year_end = 2100

year_start_list = numpy.arange(1920,2100)
year_end_list = numpy.arange(1921,2101)
year_middle_list = year_start_list+1

# create season strings
years = numpy.arange(year_start, year_end+1, 1).astype(numpy.int)
half_years_hist_rcp = numpy.arange(year_start+0.75, year_end, 1)
season_strings_hist_rcp = [str(i)+'-'+str(i+1) for i in range(year_start,year_end)]

thirty_yr_strings_hist_rcp = [str(year_start_list[i])+'-'+str(year_end_list[i]) for i in range(year_start_list.size)]
member_strings_hist_rcp = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_hist_rcp=year_end-year_start

## Open all data already saved in npy array

In [17]:
all_events_hist_rcp_3d = numpy.load('csv_files/all_events_hist_rcp_3d.npy')
print(all_events_hist_rcp_3d.shape)

(650, 40, 180)


## Calculate all return periods for all time (1 year at a time)

In [18]:
print(n_seasons_hist_rcp)

180


In [19]:
#nlat_unique = numpy.unique(cali_coast_latlon_indices[:,0]).size
return_period_values_hist_rcp = numpy.zeros((len(latlon_indices), n_seasons_hist_rcp, 3))
#print(latitude_time_values.shape)

window=1

for yr_idx in range(n_seasons_hist_rcp):
    season_strings_hist_rcp = [str(i)+'-'+str(i+1) for i in range(year_start,year_end)]
    #return_period_values_hist_rcp = numpy.zeros(( PRECT_nlat*PRECT_nlon, 3 ))
    
    for latlon_idx in range(len(latlon_indices)):

        # collect seasonal events for the specific 30 year chunk
        seasonal_events = all_events_hist_rcp_3d[latlon_idx,:,yr_idx:yr_idx+window].reshape((-1))

        return_period_values_hist_rcp[latlon_idx,yr_idx,0] = numpy.percentile(seasonal_events, return_val_perc)
        return_period_values_hist_rcp[latlon_idx,yr_idx,1] = numpy.sum(seasonal_events>return_period_values_hist_rcp[latlon_idx,yr_idx,0])
        return_period_values_hist_rcp[latlon_idx,yr_idx,2] = numpy.sum(seasonal_events>return_period_values_pic[latlon_idx,0])
    #return_period_values_hist_rcp_df = pandas.DataFrame(return_period_values_hist_rcp, columns=['Perc value hist eoc','Num hist eoc exceedances','Num PIC exceedances'])


# Using scipy library to calculate AD k-sample test

# Using R library to calculate AD k-sample test

In [48]:
import rpy2
import rpy2.robjects.packages

#rpy2.robjects.packages.isinstalled('utils')
utils = rpy2.robjects.packages.importr('utils')
utils.chooseCRANmirror(ind=1)
#utils.getCRANmirrors()

rpy2.robjects.packages.isinstalled('kSamples')
#utils.install_packages('kSamples')
kSamples = rpy2.robjects.packages.importr('kSamples')
#ad_test = rpy2.robjects.r['ad.test']

/Users/baird/anaconda/envs/pyR/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in loadNamespace(name) : there is no package called ‘kSamples’

  warnings.warn(x, RRuntimeWarning)
/Users/baird/anaconda/envs/pyR/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/Users/baird/anaconda/envs/pyR/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: There were 22 warnings (use warnings() to see them)
  warnings.warn(x, RRuntimeWarning)
/Users/baird/anaconda/envs/pyR/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)


RRuntimeError: Error in loadNamespace(name) : there is no package called ‘kSamples’


In [ ]:
print(n_seasons_hist_rcp)

In [ ]:
sig_test_values = numpy.zeros((11,n_seasons_hist_rcp))

all_events_hist_rcp_cali_coast = all_events_hist_rcp_3d.reshape((PRECT_nlat, PRECT_nlon, 40, n_seasons_hist_rcp))[ cali_coast_latlon_indices[:,0], cali_coast_latlon_indices[:,1], :,:]
#print(all_events_hist_rcp_cali_coast.shape)
all_events_hist_rcp_cali_coast = all_events_hist_rcp_cali_coast.reshape((11,3,40,n_seasons_hist_rcp))
#print(all_events_hist_rcp_cali_coast.shape)

all_events_pic_cali_coast_2d = all_events_pic.reshape((PRECT_nlat, PRECT_nlon, -1))[ cali_coast_latlon_indices[:,0], cali_coast_latlon_indices[:,1], :]
#print(all_events_pic_cali_coast_2d.shape)
all_events_pic_cali_coast_3d = all_events_pic_cali_coast_2d.reshape((11,3,-1))
#print(all_events_pic_cali_coast_3d.shape)

#all_events_pic
sig_test_crit_vals = []
for yr_idx in range(n_seasons_hist_rcp):
    print(yr_idx)
    latlon_crit_vals = []
    for lat_idx in range(all_events_hist_rcp_cali_coast.shape[0]):
        
        hist_rcp_data = all_events_hist_rcp_cali_coast[lat_idx,:,:,yr_idx].flatten()
        pic_data = all_events_pic_cali_coast_3d[lat_idx,:,:].flatten()
        
        #pic_median = numpy.nanpercentile(pic_data, 50)
        #hist_rcp_data = hist_rcp_data[hist_rcp_data>=pic_median]
        #pic_data = pic_data[pic_data>=pic_median]
        
        #hist_rcp_data = all_events_hist_rcp_cali_coast[lat_idx,:,:,yr_idx:yr_idx+30].flatten()
        d1 = rpy2.robjects.FloatVector(hist_rcp_data)
        d2 = rpy2.robjects.FloatVector(pic_data)
        sig_test_values[lat_idx,yr_idx] = ad_test(d1,d2)[6][4]

# Signif. test with moving window smoothing

# repeat the steps above to generate all possible window smoothing options

In [ ]:
sig_test_values_10 = numpy.load('csv_files/sig_test_values_window_10yrs.npy'); sig_test = '10yr'
sig_test_values_10[sig_test_values_10==0.0]=numpy.nan

sig_test_values_5 = numpy.load('csv_files/sig_test_values_window_6yrs.npy'); sig_test = '5yr'
sig_test_values_5[sig_test_values_5==0.0]=numpy.nan

sig_test_values_30 = numpy.load('csv_files/sig_test_values_window_30yrs.npy'); sig_test = '30yr'
sig_test_values_30[sig_test_values_30==0.0]=numpy.nan

sig_test_values_1 = numpy.load('csv_files/sig_test_values_no_smoothing.npy'); sig_test = '1yr'
sig_test_values_1[sig_test_values_1==0.0]=numpy.nan

sig_test_values = numpy.load('csv_files/sig_test_values_no_smoothing.npy'); sig_test = '1yr'
sig_test_values[sig_test_values==0.0]=numpy.nan

sig_vals_list = [sig_test_values_1, sig_test_values_5, sig_test_values_10, sig_test_values_30]
sig_vals_labels = ['1yr','5yr','10yr','30yr']

In [ ]:
#cf=mp.contourf(sig_test_values, levels=[0,0.1,0.5,1], extend='max')
cf=mp.contourf(sig_test_values)#, extend='max')
mp.colorbar(cf)

# ALSO calculating average return period values for 3 lon/lat grid points approaching California coast

In [ ]:
# calculate ratio of frequencies of exceedance
pic_counts = return_period_values_pic[:,1]
hist_rcp_counts = return_period_values_hist_rcp[:,:,2]

#return_values_pic_2d = return_period_values_pic[:,0].reshape(PRECT_nlat, PRECT_nlon)
#return_values_hist_rcp_2d = return_period_values_hist_rcp[:,0].reshape(PRECT_nlat, PRECT_nlon)

print(pic_counts.shape, hist_rcp_counts.shape)
count_ratio = ((hist_rcp_counts/(40)).T/(pic_counts/1798)).T
count_ratio_2d = count_ratio.reshape((PRECT_nlat, PRECT_nlon, -1))
count_ratio_2d_cali = numpy.mean( count_ratio_2d[cali_coast_latlon_indices[:,0],cali_coast_latlon_indices[:,1],:].reshape((11,3,-1)) , axis=1)

return_period_values_hist_rcp_4d = return_period_values_hist_rcp.reshape((PRECT_nlat,PRECT_nlon,-1,3))
return_period_values_hist_rcp_cali_coast = return_period_values_hist_rcp_4d[cali_coast_latlon_indices[:,0], \
                                                                           cali_coast_latlon_indices[:,1], \
                                                                           :,:]

return_period_values_hist_rcp_cali_coast = numpy.mean(return_period_values_hist_rcp_cali_coast.reshape((11,3,-1,3)), axis=1)

In [ ]:
#hist_rcp_counts_cali_coast = numpy.mean(hist_rcp_counts[cali_coast_latlon_indices[:,0],cali_coast_latlon_indices[:,1],:])
hist_rcp_counts_2d = numpy.reshape(hist_rcp_counts, (PRECT_nlat, PRECT_nlon, -1))
hist_rcp_counts_2d_cali_coast = hist_rcp_counts_2d[cali_coast_latlon_indices[:,0],cali_coast_latlon_indices[:,1],:]
hist_rcp_counts_2d_cali_coast = numpy.mean( hist_rcp_counts_2d_cali_coast.reshape((11,3,-2)), axis=1)

# Do same calculation for PIC runs

In [ ]:
print(return_period_values_pic.shape)

In [ ]:
return_period_values_pic_4d = return_period_values_pic[:,0].reshape((PRECT_nlat,PRECT_nlon))
return_period_values_pic_cali_coast = return_period_values_pic_4d[cali_coast_latlon_indices[:,0], \
                                                                           cali_coast_latlon_indices[:,1]]

return_period_values_pic_cali_coast = numpy.mean(return_period_values_pic_cali_coast.reshape((11,3,-1)), axis=1)

In [ ]:
print(return_period_values_pic_cali_coast.shape)

# Now plot for hist+RCP8.5

In [ ]:
fontsize=12

fig = mp.figure(figsize=(8,3))

ax = fig.add_subplot(111)

ax_cf = ax.contourf(half_years_hist_rcp, PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], return_period_values_hist_rcp_cali_coast[:,:,0], cmap='gist_earth_r')
#ax.set_xticks()

la_color = 'saddlebrown'
sf_color = 'darkgreen'
od_color = '#007070' # '#008080' is teal

#ax.axhline(LA_lat, c='firebrick', zorder=1)
ax.axhline(LA_lat, c=la_color, zorder=1)
ax.axhline(SF_lat, c=sf_color, zorder=1)
ax.axhline(Oroville_dam_lat, c=od_color, zorder=1)

ax.text(s='LA', x=2105, y=LA_lat, color=la_color, ha='center', va='center', fontsize=fontsize, weight='bold')
ax.text(s='SF', x=2105, y=SF_lat, color=sf_color, ha='center', va='center', fontsize=fontsize, weight='bold')
ax.text(s='OD', x=2105, y=Oroville_dam_lat, color=od_color, ha='center', va='center', fontsize=fontsize, weight='bold')

ax.set_xlim(1920,2100)
ax.set_xticks(numpy.arange(1920,2101,30))
ax.set_yticks(numpy.arange(34,43,2))

ax.tick_params(labelsize=fontsize)

ax.set_ylabel('latitude ('+degree_sign+'N)', fontsize=fontsize)
ax.set_xlabel('year', fontsize=fontsize)

divider = mpl_toolkits.axes_grid1.make_axes_locatable(ax)
cbar_ax = divider.append_axes('right', size='2.5%', pad=0.4) #fig.add_axes([0.0, -0.02, 1.0, 0.03])
cbar = fig.colorbar(ax_cf, cax=cbar_ax, orientation='vertical')
cbar.set_label('mm', fontsize=fontsize)
cbar.ax.tick_params(labelsize=fontsize)

# Calculating changes in frequency of occurrence

# ALSO calculating return period values

In [ ]:
# calculate ratio of frequencies of exceedance
pic_counts = return_period_values_pic[:,1]
hist_rcp_counts = return_period_values_hist_rcp[:,:,2]

#return_values_pic_2d = return_period_values_pic[:,0].reshape(PRECT_nlat, PRECT_nlon)
#return_values_hist_rcp_2d = return_period_values_hist_rcp[:,0].reshape(PRECT_nlat, PRECT_nlon)

print(pic_counts.shape, hist_rcp_counts.shape)
count_ratio = ((hist_rcp_counts/(40)).T/(pic_counts/1798)).T
count_ratio_2d = count_ratio.reshape((PRECT_nlat, PRECT_nlon, -1))
count_ratio_2d_cali = numpy.mean( count_ratio_2d[cali_coast_latlon_indices[:,0],cali_coast_latlon_indices[:,1],:].reshape((11,3,-1)) , axis=1)

hist_rcp_counts_2d = numpy.reshape(hist_rcp_counts, (PRECT_nlat,PRECT_nlon,-1))
pic_counts_2d = numpy.reshape(pic_counts, (PRECT_nlat,PRECT_nlon))

return_period_values_pic_2d = numpy.reshape(return_period_values_pic, (PRECT_nlat, PRECT_nlon,-1))

pic_counts_2d_cali_coast = numpy.mean(pic_counts_2d[cali_coast_latlon_indices[:,0],cali_coast_latlon_indices[:,1]].reshape((11,3)), axis=1)                                        
                                                    

In [ ]:
print(pic_counts_2d_cali_coast)

In [ ]:
class MidpointNormalize(matplotlib.colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return numpy.ma.masked_array(numpy.interp(value, x, y))

In [ ]:
# make color map
minval=0.0 # 
maxval=0.85 #
n=256
full_cmap = mp.get_cmap('RdBu')
cmap_partial = matplotlib.colors.LinearSegmentedColormap.from_list('trunc({n},{a:.2f},{b:.2f})'.format(n=full_cmap.name, a=minval, b=maxval), full_cmap(numpy.linspace(minval, maxval, n)))

In [ ]:
fontsize=12


fig = mp.figure(figsize=(8,3))

ax = fig.add_subplot(111)

contour_levels = numpy.arange(0,5.1,0.2)

ax_cf = ax.contourf(half_years_hist_rcp, \
                    PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
                    count_ratio_2d_cali, \
                    norm=MidpointNormalize(midpoint=1.0), \
                    cmap='RdBu', \
                    levels=contour_levels, \
                    extend='max')

ax_sig = ax.contourf(year_middle_list, \
                    PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
                    sig_test_values, \
                    #count_ratio_2d_cali_bool, \
                    colors='None', \
                    levels=[0.0,0.01,1.0], \
                    #levels=[0.05,1.5], \
                    hatches=['///',None], \
                    extend='lower', \
                    edgecolors='red', \
                    zorder=2)

# ax_c = ax.contour(half_years_hist_rcp, \
#                     PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
#                     count_ratio_2d_cali, \
#                     norm=MidpointNormalize(midpoint=1.0), \
#                     levels=[0,0.5,1,1.5,2,3,4,5], \
#                     colors='0.5', \
#                     linestyles=['--']*2+[':']+['-']*5, \
#                     linewidths=[1]*2+[2]+[1]*5, \
#                     zorder=3)

# new_levels = []
# for i in ax_c.levels:
#     if i in [0.5,1.5]:
#         new_levels.append('{:.1f}'.format(i))
#     else:
#         new_levels.append('{:.0f}'.format(i))
# ax_c.levels = new_levels
# mp.clabel(ax_c, fontsize=fontsize, colors='0.25')

la_color = 'saddlebrown'
sf_color = 'darkgreen'
od_color = '#007070' # '#008080' is teal

#ax.axhline(LA_lat, c='firebrick', zorder=1)
ax.axhline(LA_lat, c=la_color, zorder=2)
ax.axhline(SF_lat, c=sf_color, zorder=2)
ax.axhline(Oroville_dam_lat, c=od_color, zorder=2)

ax.text(s='LA', x=2105, y=LA_lat, color=la_color, ha='center', va='center', fontsize=fontsize, weight='bold')
ax.text(s='SF', x=2105, y=SF_lat, color=sf_color, ha='center', va='center', fontsize=fontsize, weight='bold')
ax.text(s='OD', x=2105, y=Oroville_dam_lat, color=od_color, ha='center', va='center', fontsize=fontsize, weight='bold')

ax.set_xlim(1920,2100)
ax.set_xticks(numpy.arange(1920,2101,30))
ax.set_yticks(numpy.arange(34,43,2))

ax.tick_params(labelsize=fontsize)

ax.set_ylabel('latitude ('+degree_sign+'N)', fontsize=fontsize)
ax.set_xlabel('year', fontsize=fontsize)

divider = mpl_toolkits.axes_grid1.make_axes_locatable(ax)
cbar_ax = divider.append_axes('right', size='2.5%', pad=0.4)
cbar = fig.colorbar(ax_cf, cax=cbar_ax, orientation='vertical')
cbar.set_label('ratio of frequencies', fontsize=fontsize)#, labelpad=10)
cbar.ax.tick_params(labelsize=fontsize)
cbar.set_ticks([0,1,2,3,4,5])

fig.savefig('time_latitude_plot_1yr_window.pdf', transparent=True, bbox_inches='tight')

# Apply 30 year smoothing

In [ ]:
windows = [1,6,10,30]
count_ratio_2d_cali_smoothed_list = []

for w in range(len(windows)):
    window=windows[w]
    count_ratio_2d_cali_smoothed = numpy.zeros((count_ratio_2d_cali.shape))
    for row_idx in range(count_ratio_2d_cali_smoothed.shape[0]):
        count_ratio_2d_cali_smoothed[row_idx,:] = pandas.Series(count_ratio_2d_cali[row_idx,:]).rolling(window=window, center=True).mean()
    count_ratio_2d_cali_smoothed_list.append(count_ratio_2d_cali_smoothed)

In [ ]:
fontsize=12
for value in range(len(windows)):
    
    count_ratio_2d_cali_smoothed = count_ratio_2d_cali_smoothed_list[value]

    fig = mp.figure(figsize=(8,3))

    ax = fig.add_subplot(111)

    contour_levels = numpy.arange(0,5.1,0.2)

    ax_cf = ax.contourf(half_years_hist_rcp, \
                        PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
                        count_ratio_2d_cali_smoothed, \
                        norm=MidpointNormalize(midpoint=1.0), \
                        cmap='RdBu', \
                        levels=contour_levels, \
                        extend='max')

    ax_sig = ax.contourf(year_middle_list, \
                        PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
                        sig_vals_list[value], \
                        #count_ratio_2d_cali_bool, \
                        colors='None', \
                        levels=[0.0,0.01,1.0], \
                        #levels=[0.05,1.5], \
                        hatches=['///',None], \
                        extend='lower', \
                        edgecolors='red', \
                        zorder=2)

#     ax_c = ax.contour(half_years_hist_rcp, \
#                         PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])], \
#                         count_ratio_2d_cali_smoothed, \
#                         norm=MidpointNormalize(midpoint=1.0), \
#                         levels=[0,0.5,1,1.5,2,3,4,5], \
#                         colors='0.5', \
#                         linestyles=['--']*2+[':']+['-']*5, \
#                         linewidths=[1]*2+[2]+[1]*5, \
#                         zorder=3)

#     new_levels = []
#     for i in ax_c.levels:
#         if i in [0.5,1.5]:
#             new_levels.append('{:.1f}'.format(i))
#         else:
#             new_levels.append('{:.0f}'.format(i))
#     ax_c.levels = new_levels
#     mp.clabel(ax_c, fontsize=fontsize, colors='0.25')

    la_color = 'saddlebrown'
    sf_color = 'darkgreen'
    od_color = '#007070' # '#008080' is teal

    #ax.axhline(LA_lat, c='firebrick', zorder=1)
    ax.axhline(LA_lat, c=la_color, zorder=2)
    ax.axhline(SF_lat, c=sf_color, zorder=2)
    ax.axhline(Oroville_dam_lat, c=od_color, zorder=2)

    ax.text(s='LA', x=2105, y=LA_lat, color=la_color, ha='center', va='center', fontsize=fontsize, weight='bold')
    ax.text(s='SF', x=2105, y=SF_lat, color=sf_color, ha='center', va='center', fontsize=fontsize, weight='bold')
    ax.text(s='OD', x=2105, y=Oroville_dam_lat, color=od_color, ha='center', va='center', fontsize=fontsize, weight='bold')

    ax.set_xlim(1920,2100)
    ax.set_xticks(numpy.arange(1920,2101,30))
    ax.set_yticks(numpy.arange(34,43,2))

    ax.text(s='Ratios with smoothing='+sig_vals_labels[value], x=0, y=1.03, transform=ax.transAxes, fontsize=fontsize, ha='left', va='bottom')
    ax.tick_params(labelsize=fontsize)

    ax.set_ylabel('latitude ('+degree_sign+'N)', fontsize=fontsize)
    ax.set_xlabel('year', fontsize=fontsize)

    divider = mpl_toolkits.axes_grid1.make_axes_locatable(ax)
    cbar_ax = divider.append_axes('right', size='2.5%', pad=0.4)
    cbar = fig.colorbar(ax_cf, cax=cbar_ax, orientation='vertical')
    cbar.set_label('ratio of frequencies', fontsize=fontsize)#, labelpad=10)
    cbar.ax.tick_params(labelsize=fontsize)
    cbar.set_ticks([0,1,2,3,4,5])

    fig.savefig('time_latitude_plot_'+sig_vals_labels[value]+'_window.pdf', transparent=True, bbox_inches='tight')

# Save info

In [ ]:
PRECT_lat_subset = PRECT_lat[numpy.unique(cali_coast_latlon_indices[:,0])]

In [ ]:
# convert times
time_datetime = [datetime.datetime(i,1,15) for i in year_middle_list]
time_nc = netCDF4.date2num(time_datetime, units='days since 1920-01-01', calendar='standard')

In [ ]:
# save hist clim
filename = 'time_latitude_25year_wet_events_PIC_no_smoothing.nc'

ncfile = netCDF4.Dataset(filename, 'w', format='NETCDF4')

lat_dim = ncfile.createDimension('lat', PRECT_lat_subset.size)
time_dim = ncfile.createDimension('time', len(year_middle_list))

lat_var = ncfile.createVariable('lat', 'f4', ('lat',))
time_var = ncfile.createVariable('time', 'f4', ('time',))
lat_var[:] = PRECT_lat_subset
time_var[:] = time_nc
lat_var.units = 'degrees_north'
time_var.units = 'days since 1920-01-01'
time_var.description = 'Reported for yearly (seasonal) values; no smoothing or averaging'

ratios_of_frequencies = ncfile.createVariable('time_lat_ratios', 'f4', ('lat','time'))
ratios_of_frequencies[:] = count_ratio_2d_cali
ratios_of_frequencies.units = 'unitless'
ratios_of_frequencies.description = 'Ratio of frequencies for coastal California grid points (average of 3 longitudinal values at each California coastal latitude)'

counts_hist_rcp = ncfile.createVariable('hist_rcp_counts', 'f4', ('lat','time'))
counts_hist_rcp[:] = hist_rcp_counts_2d_cali_coast
counts_hist_rcp.units = 'number of exceedances'
counts_hist_rcp.description = 'Number of exceedances of PIC event (average of 3 longitudinal values at each California coastal latitude)'

counts_pic = ncfile.createVariable('pic_counts', 'f4', ('lat'))
counts_pic[:] = pic_counts_2d_cali_coast
counts_pic.units = 'number of exceedances'
counts_pic.description = 'Number of events in PIC time series (constant value corresponding to appropriate percentile)'

return_period_vals_hist_rcp = ncfile.createVariable('return_period_vals_hist_rcp', 'f4', ('lat','time'))
return_period_vals_hist_rcp[:] = return_period_values_hist_rcp_cali_coast[:,:,0]
return_period_vals_hist_rcp.units = 'mm'
return_period_vals_hist_rcp.description = 'Return period precipitation values for hist+RCP8.5 time series (average of 3 longitudinal grid points at each California coastal latitude)'

return_period_vals_pic = ncfile.createVariable('return_period_vals_pic', 'f4', ('lat',))
return_period_vals_pic[:] = return_period_values_pic_cali_coast[:]
return_period_vals_pic.units = 'mm'
return_period_vals_pic.description = 'Return period precipitation values for PIC time series (average of 3 longitudinal grid points at each California coastal latitude)'

# sig_vals_labels = ['1yr','5yr','10yr','30yr']
ad_pvals_1yr = ncfile.createVariable('ad_pvals_1yr', 'f4', ('lat','time'))
ad_pvals_1yr[:] = sig_vals_list[0]
ad_pvals_1yr.description = 'P-values calculated from the Anderson-Darling k-samples.  Points where this field is < 0.05 are significant at the 95% conf. level; points where < 0.01 are significant at the 99% conf. level, etc.'

# sig_vals_labels = ['1yr','5yr','10yr','30yr']
ad_pvals_5yr = ncfile.createVariable('ad_pvals_5yr', 'f4', ('lat','time'))
ad_pvals_5yr[:] = sig_vals_list[1]
ad_pvals_5yr.description = 'P-values calculated from the Anderson-Darling k-samples.  Points where this field is < 0.05 are significant at the 95% conf. level; points where < 0.01 are significant at the 99% conf. level, etc.'

# sig_vals_labels = ['1yr','5yr','10yr','30yr']
ad_pvals_10yr = ncfile.createVariable('ad_pvals_10yr', 'f4', ('lat','time'))
ad_pvals_10yr[:] = sig_vals_list[2]
ad_pvals_10yr.description = 'P-values calculated from the Anderson-Darling k-samples.  Points where this field is < 0.05 are significant at the 95% conf. level; points where < 0.01 are significant at the 99% conf. level, etc.'

# sig_vals_labels = ['1yr','5yr','10yr','30yr']
ad_pvals_30yr = ncfile.createVariable('ad_pvals_30yr', 'f4', ('lat','time'))
ad_pvals_30yr[:] = sig_vals_list[3]
ad_pvals_30yr.description = 'P-values calculated from the Anderson-Darling k-samples.  Points where this field is < 0.05 are significant at the 95% conf. level; points where < 0.01 are significant at the 99% conf. level, etc.'

ncfile.history = 'Created ' + time.ctime(time.time())
ncfile.close()
print(filename, "saved")